# 01. Exploration et Validation des Données

**Objectif :** Ce notebook sert à valider le `DataManager` (tâche 2.1) et à effectuer une première analyse exploratoire des données (tâche 2.4). 

Nous allons :
1.  Tester le chargement d'un seul ticker (`AAPL`).
2.  Afficher les statistiques descriptives.
3.  Visualiser les données (OHLCV + Indicateurs) de manière interactive.
4.  Vérifier la qualité des données (NaNs, outliers).
5.  Tester le téléchargement en batch d'une liste de 10 tickers (S&P 500 + CAC 40).

In [1]:
# --- 1. Bibliothèques natives ---
import sys
import os
from pathlib import Path

# --- 2. Bibliothèques tierces ---
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# --- Configuration du Chemin ---
# Ajoute la racine du projet au PYTHONPATH pour que les imports (utils, etc.) fonctionnent
try:
    # Si exécuté depuis 'notebooks/'
    PROJECT_ROOT = Path(__file__).resolve().parent.parent
except NameError:
    # Si exécuté interactivement (cas le plus courant)
    PROJECT_ROOT = Path.cwd().parent

if str(PROJECT_ROOT) not in sys.path:
    sys.path.append(str(PROJECT_ROOT))

print(f"Project Root (ajouté au sys.path): {PROJECT_ROOT}")

# --- 3. Imports locaux du projet ---
from utils.data_manager import DataManager
from utils.data_processor import add_returns # Tâche 2.3

print("Modules importés avec succès.")

Project Root (ajouté au sys.path): c:\Users\saill\Desktop\t_project
Modules importés avec succès.


## 1. Test de Téléchargement (Ticker Unique)

Nous initialisons `DataManager` et chargeons les données pour `AAPL` sur une période de 3 ans. 

Nous forçons `use_cache=False` pour ce premier test afin de valider le processus de téléchargement et de sauvegarde. Nous activons `add_indicators=True` pour vérifier que le `DataManager` ajoute bien les colonnes.

In [2]:
dm = DataManager()

ticker = "AAPL"
start_date = "2022-01-01"
end_date = "2024-12-31"

df_aapl = dm.get_data(
    ticker=ticker,
    start_date=start_date,
    end_date=end_date,
    use_cache=False,      # Force le téléchargement pour ce test
)

df_aapl.info()
df_aapl.head()

2025-11-01 12:48:53 - utils.config_loader - INFO - Chargement des paramètres globaux depuis : c:\Users\saill\Desktop\t_project\config\settings.yaml
2025-11-01 12:48:53 - utils.data_manager - INFO - DataManager initialisé. Cache: C:\Users\saill\Desktop\t_project\data\cache. Timezone: Europe/Paris
2025-11-01 12:48:53 - utils.data_manager - INFO - Téléchargement pour AAPL (plage par défaut : 2020-01-01 à 2025-10-31)...
2025-11-01 12:48:53 - utils.data_manager - INFO - Téléchargement de AAPL (2020-01-01 à 2025-10-31, 1d)...
2025-11-01 12:48:53 - utils.data_manager - INFO - Données téléchargées avec succès pour AAPL (1466 lignes).
2025-11-01 12:48:53 - utils.data_manager - INFO - [OK] Données prêtes pour AAPL (753 lignes de 2022-01-01 à 2024-12-31).


<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 753 entries, 2022-01-03 06:00:00+01:00 to 2024-12-31 06:00:00+01:00
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   open    753 non-null    float64
 1   high    753 non-null    float64
 2   low     753 non-null    float64
 3   close   753 non-null    float64
 4   volume  753 non-null    int64  
dtypes: float64(4), int64(1)
memory usage: 35.3 KB


,open,high,low,close,volume
Date,,,,,
2022-01-03 06:00:00+01:00,174.345053,179.296091,174.227410,178.443130,104487900
2022-01-04 06:00:00+01:00,179.050948,179.354870,175.609725,176.178360,99310400
2022-01-05 06:00:00+01:00,176.090142,176.639165,171.217539,171.492050,94537600
2022-01-06 06:00:00+01:00,169.315582,171.864636,168.276357,168.629303,96904000
2022-01-07 06:00:00+01:00,169.501881,170.727385,167.678331,168.795990,86709100


## 2. Statistiques Descriptives

Un simple `.describe()` nous donne un aperçu rapide de la distribution des prix, des volumes et des indicateurs.

In [3]:
# Appliquer un formatage pour une meilleure lisibilité
print(df_aapl.describe().apply(lambda s: s.apply('{:,.2f}'.format)))

         open    high     low   close          volume
count  753.00  753.00  753.00  753.00          753.00
mean   176.17  178.04  174.48  176.36   68,077,423.51
std     29.47   29.46   29.49   29.54   28,346,260.21
min    124.26  125.99  122.44  123.28   23,234,700.00
25%    153.15  155.37  151.45  153.70   48,714,100.00
50%    171.22  172.52  169.77  171.51   62,199,000.00
75%    189.88  191.46  188.81  189.95   80,546,200.00
max    257.28  259.18  256.72  258.10  318,679,900.00


In [4]:
bbands_cols = [col for col in df_aapl.columns if 'bb' in col.upper()]

print("Colonnes 'BBands' réellement trouvées dans le DataFrame :")
print(bbands_cols)
print(df_aapl.columns)



Colonnes 'BBands' réellement trouvées dans le DataFrame :
[]
Index(['open', 'high', 'low', 'close', 'volume'], dtype='object')


## 3. Visualisation des Données (Plotly)

Nous vérifions visuellement les données OHLCV ainsi que les indicateurs ajoutés par `DataManager` :
* **Prix** + **Bandes de Bollinger** (`BBU_20_2.0`, `BBL_20_2.0`)
* **Volume**
* **MACD** (`MACD_12_26_9`, `MACDs_12_26_9`, `MACDh_12_26_9`)
* **RSI** (`RSI_14`)

*(Note: L'ATR `ATRr_14` est calculé mais n'est pas affiché ici pour plus de clarté)*

In [5]:
fig = make_subplots(
    rows=4, 
    cols=1, 
    shared_xaxes=True, 
    vertical_spacing=0.03,
    subplot_titles=('Prix (AAPL) avec Bandes de Bollinger', 'Volume', 'MACD', 'RSI'),
    row_heights=[0.5, 0.1, 0.2, 0.2]
)

# --- Rangée 1: Prix + Bandes de Bollinger ---
fig.add_trace(go.Candlestick(x=df_aapl.index,
                open=df_aapl['open'],
                high=df_aapl['high'],
                low=df_aapl['low'],
                close=df_aapl['close'],
                name='OHLC'), row=1, col=1)

# LIGNE CORRIGÉE : Utilise 'BBU_20_2.0_2.0'
fig.add_trace(go.Scatter(x=df_aapl.index, y=df_aapl['BBU_20_2.0_2.0'], 
                         line=dict(color='blue', width=1, dash='dash'), 
                         name='BB Upper'), row=1, col=1)

# LIGNE CORRIGÉE : Utilise 'BBL_20_2.0_2.0'
fig.add_trace(go.Scatter(x=df_aapl.index, y=df_aapl['BBL_20_2.0_2.0'], 
                         line=dict(color='blue', width=1, dash='dash'), 
                         name='BB Lower', fill='tonexty', fillcolor='rgba(0,0,255,0.05)'), row=1, col=1)

# --- Rangée 2: Volume ---
fig.add_trace(go.Bar(x=df_aapl.index, y=df_aapl['volume'], name='Volume', marker_color='grey'), row=2, col=1)

# --- Rangée 3: MACD ---
fig.add_trace(go.Scatter(x=df_aapl.index, y=df_aapl['MACD_12_26_9'], 
                         line=dict(color='green', width=1.5), name='MACD'), row=3, col=1)
fig.add_trace(go.Scatter(x=df_aapl.index, y=df_aapl['MACDs_12_26_9'], 
                         line=dict(color='red', width=1.5), name='Signal'), row=3, col=1)
fig.add_trace(go.Bar(x=df_aapl.index, y=df_aapl['MACDh_12_26_9'], 
                      name='Histogramme', marker_color='grey', opacity=0.5), row=3, col=1)

# --- Rangée 4: RSI ---
fig.add_trace(go.Scatter(x=df_aapl.index, y=df_aapl['RSI_14'], 
                         line=dict(color='purple', width=1.5), name='RSI'), row=4, col=1)
fig.add_hline(y=70, line_dash="dash", line_color="red", row=4, col=1)
fig.add_hline(y=30, line_dash="dash", line_color="green", row=4, col=1)

# --- Mise en Forme ---
fig.update_layout(
    height=1000, 
    title_text=f"Analyse Technique Interactive - {ticker}",
    xaxis_rangeslider_visible=False,
    legend_orientation='h',
    legend_yanchor='bottom',
    legend_y=1.01,
    legend_xanchor='left',
    legend_x=0
)

fig.show()

KeyError: 'BBU_20_2.0_2.0'

## 4. Vérification de la Qualité des Données

Nous vérifions deux points critiques :

1.  **Données Manquantes (NaNs) :** `DataManager` est censé supprimer les `NaNs` initiaux après le calcul des indicateurs. Le total des NaNs doit être 0.
2.  **Outliers (Rendements) :** Nous utilisons `data_processor.add_returns` pour calculer les rendements journaliers et nous vérifions les quantiles extrêmes (1% et 99%). Des valeurs extrêmes (> 15-20%) pourraient indiquer des problèmes de données (splits non ajustés, etc.).

In [ ]:
# 1. Vérification des NaNs
nan_count = df_aapl.isnull().sum().sum()
print(f"Total des valeurs NaN dans le DataFrame: {nan_count}")

if nan_count == 0:
    print("Vérification NaN: SUCCÈS. (Les périodes de chauffe des indicateurs ont été purgées)")
else:
    print("Vérification NaN: ÉCHEC.")
    print(df_aapl.isnull().sum())

# 2. Détection d'outliers (via les rendements)
df_aapl_returns = add_returns(df_aapl[['open', 'high', 'low', 'close', 'volume']].copy()) # Utilise une copie sans indicateurs

print("\n--- Statistiques des Rendements Journaliers (pct_return) ---")
print(df_aapl_returns['pct_return'].describe(percentiles=[.01, .05, .25, .75, .95, .99]).apply("{:,.4f}".format))

max_return = df_aapl_returns['pct_return'].abs().max()
print(f"\nRendement journalier absolu MAX: {max_return:.2%}")
if max_return > 0.25: # Seuil arbitraire de 25%
    print("AVERTISSEMENT: Détection d'un rendement journalier extrême > 25%. Vérification manuelle requise.")
else:
    print("Vérification Outliers (Rendements): SUCCÈS.")

Total des valeurs NaN dans le DataFrame: 0
Vérification NaN: SUCCÈS. (Les périodes de chauffe des indicateurs ont été purgées)

--- Statistiques des Rendements Journaliers (pct_return) ---
count    753.0000
mean       0.0006
std        0.0171
min       -0.0587
1%        -0.0445
5%        -0.0272
25%       -0.0084
50%        0.0011
75%        0.0097
95%        0.0258
99%        0.0442
max        0.0890
Name: pct_return, dtype: object

Rendement journalier absolu MAX: 8.90%
Vérification Outliers (Rendements): SUCCÈS.


## 5. Test de Téléchargement en Batch (5-10 Tickers)

Enfin, nous testons la capacité du `DataManager` à charger une liste de tickers (5 US, 5 FR). 

Pour ce test, nous utilisons `use_cache=True` (le comportement par défaut) pour vérifier que `AAPL` est bien lu depuis le cache (devrait être quasi-instantané) et que les 9 autres sont téléchargés.

In [ ]:
# Liste de tickers (5 S&P 500, 5 CAC 40)
# (Basé sur les fichiers config/markets/ que nous avons créés)
sp500_sample = ["AAPL", "MSFT", "GOOG", "TSLA", "NVDA"]
cac40_sample = ["MC.PA", "TTE.PA", "SAN.PA", "OR.PA", "AIR.PA"]
all_tickers = sp500_sample + cac40_sample

print(f"Début du test de chargement en batch pour {len(all_tickers)} tickers...")

data_store = {}
failed_tickers = []

for ticker in all_tickers:
    print(f"--- Chargement {ticker} ---")
    try:
        df = dm.get_data(
            ticker=ticker, 
            start_date=start_date, 
            end_date=end_date, 
            use_cache=True
        )
        
        if df.empty:
            print(f"ÉCHEC: Pas de données retournées pour {ticker}")
            failed_tickers.append(ticker)
        else:
            print(f"SUCCÈS: {len(df)} lignes chargées pour {ticker}")
            data_store[ticker] = df
            
    except Exception as e:
        print(f"ERREUR CRITIQUE pour {ticker}: {e}")
        failed_tickers.append(ticker)

print("\n--- RAPPORT DU TEST BATCH ---")
print(f"Téléchargements réussis: {len(data_store)} / {len(all_tickers)}")
if failed_tickers:
    print(f"Échecs: {failed_tickers}")
else:
    print("Échecs: 0")

2025-11-01 12:45:08 - utils.data_manager - INFO - [OK] Données pour AAPL chargées depuis le cache.
2025-11-01 12:45:08 - utils.data_manager - INFO - [OK] Données prêtes pour AAPL (753 lignes de 2022-01-01 à 2024-12-31).
2025-11-01 12:45:09 - utils.data_manager - WARNING - Index invalide dans C:\Users\saill\Desktop\t_project\data\cache\MSFT_1d.csv. Re-téléchargement.
2025-11-01 12:45:09 - utils.data_manager - INFO - Téléchargement pour MSFT (plage par défaut : 2020-01-01 à 2025-10-31)...
2025-11-01 12:45:09 - utils.data_manager - INFO - Téléchargement de MSFT (2020-01-01 à 2025-10-31, 1d)...


Début du test de chargement en batch pour 10 tickers...
--- Chargement AAPL ---
SUCCÈS: 753 lignes chargées pour AAPL
--- Chargement MSFT ---


2025-11-01 12:45:09 - utils.data_manager - INFO - Données téléchargées avec succès pour MSFT (1466 lignes).
2025-11-01 12:45:09 - utils.data_manager - INFO - [OK] Données prêtes pour MSFT (753 lignes de 2022-01-01 à 2024-12-31).
2025-11-01 12:45:09 - utils.data_manager - WARNING - Index invalide dans C:\Users\saill\Desktop\t_project\data\cache\GOOG_1d.csv. Re-téléchargement.
2025-11-01 12:45:09 - utils.data_manager - INFO - Téléchargement pour GOOG (plage par défaut : 2020-01-01 à 2025-10-31)...
2025-11-01 12:45:09 - utils.data_manager - INFO - Téléchargement de GOOG (2020-01-01 à 2025-10-31, 1d)...


SUCCÈS: 753 lignes chargées pour MSFT
--- Chargement GOOG ---


2025-11-01 12:45:09 - utils.data_manager - INFO - Données téléchargées avec succès pour GOOG (1466 lignes).
2025-11-01 12:45:09 - utils.data_manager - INFO - [OK] Données prêtes pour GOOG (753 lignes de 2022-01-01 à 2024-12-31).
2025-11-01 12:45:09 - utils.data_manager - INFO - [OK] Données pour TSLA chargées depuis le cache.
2025-11-01 12:45:09 - utils.data_manager - INFO - [OK] Données prêtes pour TSLA (753 lignes de 2022-01-01 à 2024-12-31).
2025-11-01 12:45:09 - utils.data_manager - WARNING - Index invalide dans C:\Users\saill\Desktop\t_project\data\cache\NVDA_1d.csv. Re-téléchargement.
2025-11-01 12:45:09 - utils.data_manager - INFO - Téléchargement pour NVDA (plage par défaut : 2020-01-01 à 2025-10-31)...
2025-11-01 12:45:09 - utils.data_manager - INFO - Téléchargement de NVDA (2020-01-01 à 2025-10-31, 1d)...


...

SUCCÈS: 768 lignes chargées pour SAN.PA
--- Chargement OR.PA ---


2025-11-01 12:45:11 - utils.data_manager - INFO - Données téléchargées avec succès pour OR.PA (1496 lignes).
2025-11-01 12:45:11 - utils.data_manager - INFO - [OK] Données prêtes pour OR.PA (768 lignes de 2022-01-01 à 2024-12-31).
2025-11-01 12:45:11 - utils.data_manager - WARNING - Index invalide dans C:\Users\saill\Desktop\t_project\data\cache\AIR.PA_1d.csv. Re-téléchargement.
2025-11-01 12:45:11 - utils.data_manager - INFO - Téléchargement pour AIR.PA (plage par défaut : 2020-01-01 à 2025-10-31)...
2025-11-01 12:45:11 - utils.data_manager - INFO - Téléchargement de AIR.PA (2020-01-01 à 2025-10-31, 1d)...


SUCCÈS: 768 lignes chargées pour OR.PA
--- Chargement AIR.PA ---


2025-11-01 12:45:11 - utils.data_manager - INFO - Données téléchargées avec succès pour AIR.PA (1496 lignes).
2025-11-01 12:45:11 - utils.data_manager - INFO - [OK] Données prêtes pour AIR.PA (768 lignes de 2022-01-01 à 2024-12-31).


SUCCÈS: 768 lignes chargées pour AIR.PA

--- RAPPORT DU TEST BATCH ---
Téléchargements réussis: 10 / 10
Échecs: 0


## Conclusion

Si toutes les cellules ci-dessus se sont exécutées sans erreur :

1.  Le `DataManager` est capable de télécharger, mettre en cache et recharger les données.
2.  Les indicateurs (`pandas-ta`) sont correctement calculés et ajoutés.
3.  Le `DataProcessor` (pour les rendements) fonctionne.
4.  Les données semblent propres (pas de NaNs, pas d'outliers évidents de splits).

Nous sommes prêts à passer à la Phase 3 : Développement des Stratégies.